In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense, Flatten
from keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers import Flatten, Dense, Dropout




In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls '/content/drive'


MyDrive


In [3]:
#Give dataset path
train_path = '/content/drive/MyDrive/archive (6)/Alzheimer_s Dataset/train'
test_path = '/content/drive/MyDrive/archive (6)/Alzheimer_s Dataset/test'

In [4]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_set = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'  # Adjust based on your dataset
)

test_set = test_datagen.flow_from_directory(
    test_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'  # Adjust based on your dataset
)

Found 5121 images belonging to 4 classes.
Found 1279 images belonging to 4 classes.


In [5]:
# Load Pretrained VGG16 Model
vgg16_base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers in the base model
for layer in vgg16_base.layers:
    layer.trainable = False

# Add custom layers on top of the base model
x = Flatten()(vgg16_base.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)  # Adding dropout for regularization
x = Dense(128, activation='relu')(x)  # Adding another dense layer
predictions = Dense(4, activation='softmax')(x)  # Output classes = 4, so using softmax activation


# Create the model
model = Model(inputs=vgg16_base.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])  # Use categorical_crossentropy for multi-class classification

58889256/58889256 [==============================] - 0s 0us/step


In [6]:

# Fine-tune the model
history = model.fit_generator(
    train_set,
    steps_per_epoch=len(train_set),
    epochs=30,
    validation_data=test_set,
    validation_steps=len(test_set)
)
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_set, steps=len(test_set))

print("Test Accuracy:", test_accuracy)




<ipython-input-6-02ae72a1df19>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/30
161/161 [==============================] - 1970s 12s/step - loss: 1.1555 - accuracy: 0.4952 - val_loss: 0.9350 - val_accuracy: 0.5113
Epoch 2/30
161/161 [==============================] - 29s 181ms/step - loss: 0.9575 - accuracy: 0.5296 - val_loss: 0.9252 - val_accuracy: 0.5410
Epoch 3/30
161/161 [==============================] - 30s 184ms/step - loss: 0.9133 - accuracy: 0.5382 - val_loss: 0.9001 - val_accuracy: 0.5465
Epoch 4/30
161/161 [==============================] - 30s 184ms/step - loss: 0.8958 - accuracy: 0.5505 - val_loss: 0.8755 - val_accuracy: 0.5895
Epoch 5/30
161/161 [==============================] - 29s 181ms/step - loss: 0.8608 - accuracy: 0.5804 - val_loss: 0.8666 - val_accuracy: 0.6145
Epoch 6/30
161/161 [==============================] - 34s 212ms/step - loss: 0.8799 - accuracy: 0.5606 - val_loss: 0.8834 - val_accuracy: 0.6255
Epoch 7/30
161/161 [==============================] - 29s 181ms/step - loss: 0.8997 - accuracy: 0.5474 - val_loss: 0.8811 - val_ac

In [12]:
model.save('/content/drive/MyDrive/vgg16beforeedit.h5')